<a href="https://colab.research.google.com/github/creativesar/Rock-paper-scissors/blob/main/Rock_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import os

# Colors for terminal text
class Colors:
    BLACK = '\033[30m'      # Added black color
    BG_WHITE = '\033[47m'   # Added white background for contrast with black text
    RESET = '\033[0m'
    BOLD = '\033[1m'

# Game elements with emoji
CHOICES = {
    "rock": "🪨",
    "paper": "📄",
    "scissors": "✂️",
    "fire": "🔥",  # New special choice
    "water": "💧"  # New special choice
}

# Power-ups that player can earn
POWER_UPS = {
    "double": "⚡ Double Points",
    "block": "🛡️ Block Loss",
    "peek": "👁️ See Computer's Move",
    "swap": "🔄 Swap Choices",
    "lucky": "🍀 Lucky Charm"
}

# Special moves rules
SPECIAL_RULES = {
    "fire": ["paper", "scissors"],  # Fire beats paper and scissors
    "water": ["rock", "fire"]       # Water beats rock and fire
}

def clear_screen():
    """Clear the terminal screen."""
    os.system('cls' if os.name == 'nt' else 'clear')

def get_user_choice(power_ups, special_unlocked):
    """Get the user's choice with power-up option."""
    print(f"{Colors.BLACK}Choose your move:{Colors.RESET}")

    # Show regular choices
    for i, (choice, emoji) in enumerate(CHOICES.items(), 1):
        # Only show special choices if unlocked
        if choice in ["fire", "water"] and not special_unlocked:
            continue
        print(f"{Colors.BLACK}{i}. {choice.capitalize()} {emoji}{Colors.RESET}")

    # Show power-ups if available
    if any(power_ups.values()):
        print(f"\n{Colors.BLACK}Power-ups available:{Colors.RESET}")
        power_index = len([c for c in CHOICES if c not in ["fire", "water"] or special_unlocked]) + 1
        for power, name in POWER_UPS.items():
            if power_ups[power]:
                print(f"{Colors.BLACK}{power_index}. {name}{Colors.RESET}")
                power_index += 1

    while True:
        choice = input(f"\n{Colors.BOLD}{Colors.BLACK}Your choice: {Colors.RESET}").lower()

        # Handle numeric input
        if choice.isdigit():
            num = int(choice)
            available_choices = [c for c in CHOICES if c not in ["fire", "water"] or special_unlocked]
            if 1 <= num <= len(available_choices):
                return available_choices[num-1], None

            # Check if it's a power-up
            power_index = len(available_choices) + 1
            for power in POWER_UPS:
                if power_ups[power] and num == power_index:
                    return None, power
                if power_ups[power]:
                    power_index += 1

        # Handle text input
        elif choice in CHOICES and (choice not in ["fire", "water"] or special_unlocked):
            return choice, None
        elif choice in power_ups and power_ups[choice]:
            return None, choice

        print(f"{Colors.BLACK}Invalid choice. Try again!{Colors.RESET}")

def play_game(scores, streak, power_ups, special_unlocked, combo_meter):
    """Play a single round with special features."""
    clear_screen()
    print(f"{Colors.BOLD}{Colors.BG_WHITE}{Colors.BLACK} ✨ ROCK PAPER SCISSORS ULTIMATE ✨ {Colors.RESET}")
    print(f"{Colors.BLACK}Score: You {scores['user']} - {scores['computer']} Computer | Streak: {streak} | Combo: {combo_meter}/5{Colors.RESET}")

    # Handle peek power-up first
    computer_choice = random.choice([c for c in CHOICES if c not in ["fire", "water"] or random.random() < 0.2])
    if power_ups["peek"]:
        use_peek = input(f"\n{Colors.BLACK}Use your Peek power-up to see computer's move? (y/n): {Colors.RESET}").lower() == 'y'
        if use_peek:
            print(f"{Colors.BLACK}Computer will choose: {computer_choice.upper()} {CHOICES[computer_choice]}{Colors.RESET}")
            power_ups["peek"] = False

    # Get user choice
    user_choice, power_up = get_user_choice(power_ups, special_unlocked)

    # Handle power-ups
    double_points = False
    block_loss = False
    lucky_charm = False

    if power_up == "double":
        print(f"{Colors.BLACK}Double Points activated!{Colors.RESET}")
        double_points = True
        power_ups["double"] = False
        user_choice, _ = get_user_choice({k: False for k in power_ups}, special_unlocked)

    elif power_up == "block":
        print(f"{Colors.BLACK}Block Loss activated!{Colors.RESET}")
        block_loss = True
        power_ups["block"] = False
        user_choice, _ = get_user_choice({k: False for k in power_ups}, special_unlocked)

    elif power_up == "swap":
        print(f"{Colors.BLACK}Swap Choices activated! Computer's choice changed!{Colors.RESET}")
        # Change computer's choice to something else
        old_choice = computer_choice
        while computer_choice == old_choice:
            computer_choice = random.choice([c for c in CHOICES if c not in ["fire", "water"] or random.random() < 0.2])
        power_ups["swap"] = False
        user_choice, _ = get_user_choice({k: False for k in power_ups}, special_unlocked)

    elif power_up == "lucky":
        print(f"{Colors.BLACK}Lucky Charm activated! 50% chance to win automatically!{Colors.RESET}")
        lucky_charm = True
        power_ups["lucky"] = False
        user_choice, _ = get_user_choice({k: False for k in power_ups}, special_unlocked)

    # Show choices with cool animation
    print(f"\n{Colors.BLACK}You chose: ", end="")
    for char in f"{user_choice.upper()} {CHOICES[user_choice]}":
        print(char, end="", flush=True)
        time.sleep(0.05)
    print(f"{Colors.RESET}")

    # Computer choice with suspense
    print(f"{Colors.BLACK}Computer is choosing", end="")
    for _ in range(3):
        time.sleep(0.3)
        print(".", end="", flush=True)
    print(f" {computer_choice.upper()} {CHOICES[computer_choice]}{Colors.RESET}")

    # Determine winner with special rules
    if lucky_charm and random.random() < 0.5:
        result = "user"
        message = f"{Colors.BLACK}🍀 Lucky Charm worked! Automatic win!{Colors.RESET}"
        streak = streak + 1 if streak >= 0 else 1
        combo_meter += 1
    elif user_choice == computer_choice:
        result = "tie"
        message = f"{Colors.BLACK}It's a tie!{Colors.RESET}"
        streak = 0
        combo_meter = 0
    else:
        # Check special moves first
        if user_choice in SPECIAL_RULES and computer_choice in SPECIAL_RULES[user_choice]:
            result = "user"
            message = f"{Colors.BLACK}You win with special move! {user_choice.capitalize()} beats {computer_choice}!{Colors.RESET}"
            streak = streak + 1 if streak >= 0 else 1
            combo_meter += 1
        elif computer_choice in SPECIAL_RULES and user_choice in SPECIAL_RULES[computer_choice]:
            if block_loss:
                result = "tie"
                message = f"{Colors.BLACK}Loss blocked by your shield!{Colors.RESET}"
                streak = 0
                combo_meter = 0
            else:
                result = "computer"
                message = f"{Colors.BLACK}Computer wins with special move! {computer_choice.capitalize()} beats {user_choice}!{Colors.RESET}"
                streak = streak - 1 if streak <= 0 else -1
                combo_meter = 0
        # Then check regular rules
        elif ((user_choice == "rock" and computer_choice == "scissors") or
              (user_choice == "paper" and computer_choice == "rock") or
              (user_choice == "scissors" and computer_choice == "paper")):
            result = "user"
            message = f"{Colors.BLACK}You win! {user_choice.capitalize()} beats {computer_choice}!{Colors.RESET}"
            streak = streak + 1 if streak >= 0 else 1
            combo_meter += 1
        else:
            if block_loss:
                result = "tie"
                message = f"{Colors.BLACK}Loss blocked by your shield!{Colors.RESET}"
                streak = 0
                combo_meter = 0
            else:
                result = "computer"
                message = f"{Colors.BLACK}Computer wins! {computer_choice.capitalize()} beats {user_choice}!{Colors.RESET}"
                streak = streak - 1 if streak <= 0 else -1
                combo_meter = 0

    # Update score
    if result != "tie":
        if result == "user" and double_points:
            scores[result] += 2
            print(f"{Colors.BLACK}+2 points from Double Points!{Colors.RESET}")
        else:
            scores[result] += 1

    print(f"\n{message}")

    # Check for combo meter full
    if combo_meter >= 5:
        print(f"{Colors.BG_WHITE}{Colors.BOLD}{Colors.BLACK} COMBO MASTER! {Colors.RESET}")
        print(f"{Colors.BLACK}You filled the combo meter! Special moves unlocked!{Colors.RESET}")
        special_unlocked = True
        combo_meter = 0

    # Award power-up on streak
    if streak == 3:
        available = [p for p, active in power_ups.items() if not active]
        if available:
            power_up = random.choice(available)
            power_ups[power_up] = True
            print(f"{Colors.BLACK}🎁 3-Win Streak! You earned: {POWER_UPS[power_up]}{Colors.RESET}")

    return streak, special_unlocked, combo_meter

def show_intro():
    """Show a cool intro animation."""
    clear_screen()
    intro_text = "ROCK PAPER SCISSORS ULTIMATE"
    print("\n\n")
    for char in intro_text:
        print(f"{Colors.BLACK}{char}{Colors.RESET}", end="", flush=True)
        time.sleep(0.1)
    print("\n\n")
    time.sleep(0.5)

    print(f"{Colors.BLACK}Get ready for the ultimate game!{Colors.RESET}")
    print(f"{Colors.BLACK}Win 5 times in a row to unlock special moves: {CHOICES['fire']} Fire and {CHOICES['water']} Water!{Colors.RESET}")
    print(f"{Colors.BLACK}Fire beats Paper and Scissors!{Colors.RESET}")
    print(f"{Colors.BLACK}Water beats Rock and Fire!{Colors.RESET}")
    time.sleep(2)

def main():
    """Main game function with special features."""
    show_intro()

    scores = {"user": 0, "computer": 0}
    streak = 0  # Positive for player streak, negative for computer streak
    power_ups = {"double": False, "block": False, "peek": False, "swap": False, "lucky": False}
    special_unlocked = False  # Special moves unlocked
    combo_meter = 0  # Tracks consecutive wins for special moves

    # Give initial power-up
    power_ups["peek"] = True
    print(f"{Colors.BLACK}You start with: {POWER_UPS['peek']}{Colors.RESET}")
    time.sleep(1.5)

    while True:
        streak, special_unlocked, combo_meter = play_game(scores, streak, power_ups, special_unlocked, combo_meter)

        print(f"\n{Colors.BLACK}Play again? (yes/no){Colors.RESET}")
        if input(f"{Colors.BOLD}{Colors.BLACK}> {Colors.RESET}").lower() != "yes":
            clear_screen()
            print(f"{Colors.BG_WHITE}{Colors.BLACK} ===== FINAL SCORE ===== {Colors.RESET}")
            print(f"{Colors.BLACK}You: {scores['user']} | Computer: {scores['computer']}{Colors.RESET}")

            if scores['user'] > scores['computer']:
                print(f"{Colors.BLACK}🏆 You are the ultimate champion!{Colors.RESET}")
            elif scores['computer'] > scores['user']:
                print(f"{Colors.BLACK}💔 Better luck next time!{Colors.RESET}")
            else:
                print(f"{Colors.BLACK}🤝 It's a tie!{Colors.RESET}")

            print(f"{Colors.BLACK}Thanks for playing!{Colors.RESET}")
            break

if __name__ == "__main__":
    main()




ROCK PAPER SCISSORS ULTIMATE


Get ready for the ultimate game!
Win 5 times in a row to unlock special moves: 🔥 Fire and 💧 Water!
Fire beats Paper and Scissors!
Water beats Rock and Fire!
You start with: 👁️ See Computer's Move
 ✨ ROCK PAPER SCISSORS ULTIMATE ✨ 
Score: You 0 - 0 Computer | Streak: 0 | Combo: 0/5
